In [6]:
import json
with open('./historical_aqi_weather_data.json', "r") as f:
    data = json.load(f)


In [7]:
import json
import pandas as pd

# Load JSON file
with open('./historical_aqi_weather_data.json', 'r') as f:
    data = json.load(f)

records = []

for item in data:
    weather_data = item['weather']
    aqi_info = item['aqi']
    aqi_list = aqi_info['list']

    if aqi_list:  # AQI data is available
        aqi_data = aqi_list[0]
        components = aqi_data['components']
        record = {
            'aqi_index': aqi_data['main']['aqi'],
            'aqi_timestamp': aqi_data['dt'],
            'co': components['co'],
            'no': components['no'],
            'no2': components['no2'],
            'o3': components['o3'],
            'so2': components['so2'],
            'pm2_5': components['pm2_5'],
            'pm10': components['pm10'],
            'nh3': components['nh3'],
        }
    else:  # AQI data is missing
        record = {
            'aqi_index': None,
            'aqi_timestamp': None,
            'co': None,
            'no': None,
            'no2': None,
            'o3': None,
            'so2': None,
            'pm2_5': None,
            'pm10': None,
            'nh3': None,
        }

    # Add weather data
    record.update({
        'temperature_2m': weather_data['temperature_2m'],
        'relative_humidity_2m': weather_data['relative_humidity_2m'],
        'precipitation': weather_data['precipitation'],
        'wind_speed_10m': weather_data['wind_speed_10m'],
        'wind_direction_10m': weather_data['wind_direction_10m'],
        'surface_pressure': weather_data['surface_pressure'],
        'dew_point_2m': weather_data['dew_point_2m'],
        'apparent_temperature': weather_data['apparent_temperature'],
        'shortwave_radiation': weather_data['shortwave_radiation'],
        'et0_fao_evapotranspiration': weather_data['et0_fao_evapotranspiration'],
    })

    records.append(record)

# Convert to DataFrame
df = pd.DataFrame(records)

# Save to CSV
df.to_csv('aqi_weather_data.csv', index=False)
print("CSV file created: aqi_weather_data.csv")


CSV file created: aqi_weather_data.csv


In [8]:
print(df.head())

   aqi_index  aqi_timestamp       co    no    no2     o3    so2  pm2_5  \
0        4.0   1.729624e+09  1201.63  0.01  45.24  41.13   8.82  68.43   
1        4.0   1.729627e+09  1388.55  0.05  57.58  26.46   8.94  71.22   
2        5.0   1.729631e+09  1468.66  0.06  58.26  21.99  10.01  76.47   
3        5.0   1.729634e+09  1375.20  0.01  48.67  26.82  10.73  79.89   
4        4.0   1.729638e+09  1001.36  0.00  30.85  43.99   8.82  72.51   

     pm10    nh3  temperature_2m  relative_humidity_2m  precipitation  \
0  112.67  11.78            27.3                  81.0            0.0   
1  113.28  10.39            26.8                  85.0            0.0   
2  117.74  10.64            26.2                  90.0            0.0   
3  121.13  11.02            26.0                  94.0            0.0   
4  111.48   8.87            26.0                  94.0            0.0   

   wind_speed_10m  wind_direction_10m  surface_pressure  dew_point_2m  \
0            11.7               314.0      

In [9]:
print(df)

      aqi_index  aqi_timestamp       co    no    no2     o3    so2  pm2_5  \
0           4.0   1.729624e+09  1201.63  0.01  45.24  41.13   8.82  68.43   
1           4.0   1.729627e+09  1388.55  0.05  57.58  26.46   8.94  71.22   
2           5.0   1.729631e+09  1468.66  0.06  58.26  21.99  10.01  76.47   
3           5.0   1.729634e+09  1375.20  0.01  48.67  26.82  10.73  79.89   
4           4.0   1.729638e+09  1001.36  0.00  30.85  43.99   8.82  72.51   
...         ...            ...      ...   ...    ...    ...    ...    ...   
4333        4.0   1.745251e+09   125.66  0.00   0.08  89.66   0.22  36.13   
4334        4.0   1.745255e+09   125.84  0.00   0.08  89.68   0.23  36.91   
4335        4.0   1.745608e+09   113.41  0.00   0.11  61.93   0.43  38.76   
4336        4.0   1.745653e+09   113.63  0.01   0.05  72.76   0.43  44.26   
4337        4.0   1.745654e+09   113.63  0.01   0.05  72.76   0.43  44.26   

        pm10    nh3  temperature_2m  relative_humidity_2m  precipitation  \